In [1]:
import numpy as np
import torch
from model import RNN

torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### load dataset ###

def load_data(mode='train'):
    data = np.loadtxt('./data_{}.txt'.format(mode), dtype='str')
    inputs = data[:,:2]
    labels = data[:,2]

    def strs2mat(strings):
        num = strings.shape[0]
        mat = []
        for i in range(num):
            mat.append([*strings[i]])
        return mat

    inputs_ = np.transpose(np.array([strs2mat(inputs[:,0]), strs2mat(inputs[:,1])]), (1,2,0)).astype('int')
    labels_ = np.array(strs2mat(labels))[:,:,np.newaxis].astype('int')

    return inputs_, labels_

inputs_train, labels_train = load_data(mode='train')
inputs_test, labels_test = load_data(mode='test')


### define specification ###

def specficiation(inputs, labels):
    num = inputs.shape[0]
    success = True
        
    # each number should be either 0 or 1
    temp = (inputs == 0).astype(int) + (inputs == 1).astype(int)
    if np.sum(temp < 1) > 0:
        print('not all elements are 0 or 1')
        success = False
        
    temp = (labels == 0).astype(int) + (labels == 1).astype(int)
    if np.sum(temp < 1) > 0:
        print('not all elements are 0 or 1')
        success = False
            
    # labels should be sum of inputs: sum(2**i*a[i]) + sum(2**i*b[i]) = sum(2**i*c[i])
    seq_len = inputs_train.shape[1]
    inputs_num = np.sum(np.sum(inputs_train * 2**np.arange(seq_len,)[np.newaxis,:,np.newaxis], axis=1), axis=1)
    labels_num = np.sum(np.sum(labels_train * 2**np.arange(seq_len,)[np.newaxis,:,np.newaxis], axis=1), axis=1)
    err = np.sum(1-(inputs_num == labels_num))
    if err > 0:
        print('sum of inputs string is not output string')
        success = False
        
    if success == True:
        print("Verficaition succeeded :-)")
    else:
        print("Verficaition failed :-(")
        
        
        

cuda


In [2]:
# verify dataset

specficiation(inputs_train, labels_train)

Verficaition succeeded :-)


In [3]:
# verify model

import warnings
warnings.filterwarnings("ignore")

inputs_train = torch.tensor(inputs_train, dtype=torch.float64, requires_grad=True).to(device)

# load model 
model = RNN(hidden_dim=2, device=device).to(device)
model.load_state_dict(torch.load('./model'))

pred_label_train = np.round(model(inputs_train).cpu().detach().numpy()).astype(int)
inputs_train = inputs_train.cpu().detach().numpy().astype(int)

specficiation(inputs_train, pred_label_train)

Verficaition succeeded :-)
